In [33]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

# Set random seed
np.random.seed(0)

In [2]:
%store -r data

In [35]:
#data

In [4]:
data.Activity.unique() #array([1, 2, 3, 4, 5, 6])

array([1, 2, 3, 4, 5, 6])

In [5]:
# New Column 'isTrain' == True for a row if random number generated (between 0 and 1) is less than or equal to .75
# and false otherwise

data['is_train'] = np.random.uniform(0, 1, len(data)) <= .75
data.head()

,RA_xacc,RA_yacc,RA_zacc,RA_xgyro,RA_ygyro,RA_zgyro,RA_xmag,RA_ymag,RA_zmag,LA_xacc,...,LA_ygyro,LA_zgyro,LA_xmag,LA_ymag,LA_zmag,Subject,Activity,rowData,Frequence,is_train
0,-0.60784,1.3908,9.7056,-0.003695,-0.007452,0.001598,-0.33699,-0.10005,-0.95706,-0.39088,...,0.003125,-0.011263,-0.067876,0.79872,-0.21041,1,1,1,1,True
1,-0.60314,1.4491,9.6961,0.014373,0.003043,-0.009156,-0.33723,-0.10084,-0.95680,-0.35910,...,0.004322,-0.004176,-0.067793,0.79870,-0.20877,1,1,2,2,True
2,-0.61312,1.5000,9.6842,-0.000147,-0.000306,0.009623,-0.33727,-0.10043,-0.95746,-0.35433,...,-0.006623,-0.007716,-0.067630,0.79884,-0.21053,1,1,3,3,True
3,-0.60561,1.4297,9.6790,0.024193,-0.007612,-0.013744,-0.33686,-0.09966,-0.95693,-0.36155,...,-0.004702,0.001095,-0.066574,0.79853,-0.20959,1,1,4,4,True
4,-0.60825,1.4976,9.6769,0.008742,-0.011820,0.007711,-0.33740,-0.10030,-0.95747,-0.36405,...,-0.012473,-0.020107,-0.066845,0.79829,-0.20997,1,1,5,5,True


In [6]:
# 2 dataFrames: for Training and Testing
train, test = data[data['is_train']==True], data[data['is_train']==False]
print('Number of rows the training set:', len(train))
print('Number of rows in the test set:',len(test))

Number of rows the training set: 270020
Number of rows in the test set: 89980


In [26]:
# list of the feature column's names
features = data.columns[:18]
features

Index(['RA_xacc', 'RA_yacc', 'RA_zacc', 'RA_xgyro', 'RA_ygyro', 'RA_zgyro',
       'RA_xmag', 'RA_ymag', 'RA_zmag', 'LA_xacc', 'LA_yacc', 'LA_zacc',
       'LA_xgyro', 'LA_ygyro', 'LA_zgyro', 'LA_xmag', 'LA_ymag', 'LA_zmag'],
      dtype='object')

In [27]:
#target
y = pd.factorize(train['Activity'])[0]+1
y

array([1, 1, 1, ..., 6, 6, 6])

In [28]:
# Create random forest Classifier
clf = RandomForestClassifier(n_jobs=2, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (activities)
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [10]:
# Apply the Classifier we trained to the test data
clf.predict(test[features])

array([1, 1, 1, ..., 6, 6, 6])

In [11]:
# predicted probabilities of the first 10 rows
clf.predict_proba(test[features])[0:10]

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
predictions = clf.predict(test[features])
predictions

array([1, 1, 1, ..., 6, 6, 6])

In [13]:
test['Activity'].head()

7     1
8     1
10    1
13    1
17    1
Name: Activity, dtype: int64

In [14]:
# Confusion matrix
pd.crosstab(test['Activity'], predictions, rownames=['Actual Activity'], colnames=['Predicted Activity'])

Predicted Activity,1,2,3,4,5,6
Actual Activity,,,,,,
1,14911,0,5,0,0,0
2,0,15210,3,2,0,0
3,0,2,15091,5,0,0
4,0,6,3,15133,0,0
5,0,2,2,0,14715,1
6,0,0,0,1,1,14887


In [15]:
# List of the features and their importance scores
list(zip(train[features], clf.feature_importances_))

[('RA_xacc', 0.037605586691088648),
 ('RA_yacc', 0.067038783131325705),
 ('RA_zacc', 0.048890845712373884),
 ('RA_xgyro', 0.0010758749426464829),
 ('RA_ygyro', 0.005857574344025401),
 ('RA_zgyro', 0.0050328401238300567),
 ('RA_xmag', 0.13167061478426195),
 ('RA_ymag', 0.069240935757662797),
 ('RA_zmag', 0.032293003622593526),
 ('LA_xacc', 0.062957947249207627),
 ('LA_yacc', 0.054915913487286569),
 ('LA_zacc', 0.099116548961367387),
 ('LA_xgyro', 0.00076310691839094873),
 ('LA_ygyro', 0.001336653797114011),
 ('LA_zgyro', 0.0021655623895494554),
 ('LA_xmag', 0.094064141109173827),
 ('LA_ymag', 0.19065593407649023),
 ('LA_zmag', 0.095318132901611494)]

In [34]:
print(classification_report(test['Activity'],predictions))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00     14916
          2       1.00      1.00      1.00     15215
          3       1.00      1.00      1.00     15098
          4       1.00      1.00      1.00     15142
          5       1.00      1.00      1.00     14720
          6       1.00      1.00      1.00     14889

avg / total       1.00      1.00      1.00     89980



In [30]:
%store train
%store test
%store y

Stored 'train' (DataFrame)
Stored 'test' (DataFrame)
Stored 'y' (ndarray)
